In [11]:
import requests
import os
import shutil
import json
import pandas as pd
import yaml

import utils, request_models

import warnings
warnings.filterwarnings('ignore')


In [ ]:
######## File paths ########
with open("../configs/self-driving-demo.yaml", "r") as f:
    config = yaml.safe_load(f)

protein_file_path = config['paths']['protein_file_path']
diffdock_output_dir = config['paths']['diffdock_output_dir']
dsmbind_input_dir = config['paths']['diffdock_output_dir']

starting_molecule_csv = config['paths']['starting_molecule_csv']
molmim_generated_csv = config['paths']['molmim_generated_csv']
dsmbind_predictions_csv = config['paths']['dsmbind_predictions_csv']

utils.prepare_output_directory(diffdock_output_dir)
utils.prepare_output_directory(dsmbind_input_dir)

# Get folded protein
folded_protein = utils.file_to_json_compatible_string(protein_file_path)

# Get starting molecules
df_starting_molecules = pd.read_csv(starting_molecule_csv)

# Round 0
print("Round 0")
molecule_name = df_starting_molecules['Molecules'][0]
molecule = df_starting_molecules['Smiles'][0]

# Molecular Generation with MolMIM
molmim_response = request_models.call_molmim(molecule)
generated_ligands = '\n'.join([v['smiles'] for v in molmim_response['generated']])
utils.update_dataframe_molmim_generated_molecules(molmim_response['generated'], molecule_name)

# Protein-Ligand Docking with DiffDock
diffdock_response = request_models.call_diffdock(folded_protein, generated_ligands)
utils.create_diffdock_outputs_dsmbind_inputs(molecule_name, diffdock_response)

# Binding Affinity with DSMBind
os.system("python /workspace/bionemo/examples/molecule/dsmbind/infer.py")


Round 0


In [3]:

df_molmim = pd.read_csv(molmim_generated_csv) 
df_dsmbind = pd.read_csv(dsmbind_predictions_csv)
df_joined = pd.concat([df_molmim, df_dsmbind], axis=1)

df_joined.to_csv('../data/results.csv')

In [8]:
if len(df_joined[df_joined['DSMBind_predictions'] < -1600000]) < 5:
    print("yes")


yes


In [10]:
df_joined

,generated_compound_index,generated_smiles,molmim_qed_score,starting_molecule,DSMBind_predictions
0,0,CC(C)(C)C(NC(=O)C(F)(F)F)C(=O)NCCc1cccs1,0.867914,Nirmatrelvir,-1491505.625
1,1,CC(C)(C)C(NC(=O)C(F)(F)F)C(=O)N1CCC(Cc2cccnc2)CC1,0.866885,Nirmatrelvir,-1599875.125
2,2,CC(C)(C)C(NC(=O)C(F)(F)F)C(=O)N1CC2CC1(c1nc[nH...,0.852227,Nirmatrelvir,-1583344.875
3,3,CC(C)(C)C(NC(=O)C(F)(F)F)C(=O)N1CC2CC1(C)C2,0.849075,Nirmatrelvir,-1626077.750
4,4,CC(C)(C)C(NC(=O)C(F)(F)F)C(=O)N1CCO[C@@H](C2CC...,0.848105,Nirmatrelvir,-1756441.375
